# Qiskit basics

[Qiskit Documentation](https://qiskit.org/documentation/index.html)



In [ ]:
# !pip install qiskit

In [ ]:
import qiskit
qiskit.__version__

'0.21.0'

## Quantum Circuits




### Encoding an Input

We need classical bits to store the measurements of our qubits

In [ ]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(3, 3) # 4 qubits and 3 bits (the last one optional)
qc.draw()

q_0: 
     
q_1: 
     
q_2: 
     
c: 3/

A "measure" operation is needed to measure the qubits and record results

In [ ]:
qc.measure([0,1,2], [0,1,2])  # the qubits 0, 1 & 2 will be measured to bits 0, 1 & 2 respectively
qc.draw()

┌─┐      
q_0: ┤M├──────
     └╥┘┌─┐   
q_1: ─╫─┤M├───
      ║ └╥┘┌─┐
q_2: ─╫──╫─┤M├
      ║  ║ └╥┘
c: 3/═╩══╩══╩═
      0  1  2

Simulators are standards computers simulating what a real quantum computer would do

Feasible simulations have up to ~30 qubits

In [ ]:
from qiskit.providers.aer import AerSimulator
sim = AerSimulator()

The default number of times to repeat a circuit in Qiskit is 1024

By convention qubits always start in the state 0

In [ ]:
job = sim.run(qc)     # contais info about the experiment
result = job.result() # get its result

# interpret the results as a "counts" dictionary
result.get_counts() # the keys are bit-strings, the values are the number of times that bit-string was measured.

{'000': 1024}

#### X-gate (NOT)

In [ ]:
qc = QuantumCircuit(3, 3)
qc.x([0,1]) # perform X-gates on qubits 0 & 1
qc.measure([0,1,2], [0,1,2])
qc.draw()

┌───┐┌─┐   
q_0: ┤ X ├┤M├───
     ├───┤└╥┘┌─┐
q_1: ┤ X ├─╫─┤M├
     └┬─┬┘ ║ └╥┘
q_2: ─┤M├──╫──╫─
      └╥┘  ║  ║ 
c: 3/══╩═══╩══╩═
       2   0  1

In [ ]:
job = sim.run(qc)
result = job.result()
result.get_counts()

{'011': 1024}

The result is 011 (or 3 in decimal)

Encoding number 4:

In [ ]:
qc = QuantumCircuit(3, 3)
qc.x([2]) # perform X-gates on qubit 0
qc.measure([0,1,2], [0,1,2])
qc.draw()

┌─┐      
q_0: ─────┤M├──────
          └╥┘┌─┐   
q_1: ──────╫─┤M├───
     ┌───┐ ║ └╥┘┌─┐
q_2: ┤ X ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2

In [ ]:
job = sim.run(qc)
result = job.result()
result.get_counts()

{'100': 1024}

Encoding number 6:

In [ ]:
qc = QuantumCircuit(3, 3)
qc.x([1,2]) # perform X-gates on qubits 1 & 2
qc.measure([0,1,2], [0,1,2])
qc.draw()

┌─┐      
q_0: ─────┤M├──────
     ┌───┐└╥┘┌─┐   
q_1: ┤ X ├─╫─┤M├───
     ├───┤ ║ └╥┘┌─┐
q_2: ┤ X ├─╫──╫─┤M├
     └───┘ ║  ║ └╥┘
c: 3/══════╩══╩══╩═
           0  1  2

In [ ]:
job = sim.run(qc)
result = job.result()
result.get_counts()

{'110': 1024}

### Adding with quantum circuits

The half adder needs to do:

0+0 = 00

0+1 = 01

1+0 = 01

1+1 = 10

This operation is classically represented by an XOR gate

In quantum computers, the job of the XOR gate is done by the 'controlled-NOT gate', a.k.a 'CNOT'

Let's creat a quantum circuit with 2 qubits and 2 classical bits yo calculate the rightmost output bit

In [ ]:
qc = QuantumCircuit(2, 2)
qc.x(0)             # perform X-gates on qubit 0
qc.cx(0,1)          # CNOT controlled by qubit 0 and targeting qubit 1
qc.measure([0,1], [0,1])
display(qc.draw())  # display a drawing of the circuit

job = sim.run(qc)
result = job.result()
print("Result: ", result.get_counts())

┌───┐     ┌─┐   
q_0: ┤ X ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

Result:  {'11': 1024}


Write the result on a different pair of qubits, using two CNOTs and writing the output to a new qubit which we know will be in the state 0.

To calculate the left output bit, we could just look at whether both of the inputs are 1. If they are — and only if they are — we need to do a NOT gate on qubit 3. That will flip it to the required value of 1 for this case only, giving us the output we need.

We need a new gate: a CNOT but controlled on two qubits. This will perform a NOT on the target qubit only when both controls are in state 1. This new gate is called the Toffoli.

In [ ]:
test_qc = QuantumCircuit(4, 2)

# First, our circuit should encode an input (here '11')
test_qc.x(0)
test_qc.x(1)

# Next, it should carry out the adder circuit we created
test_qc.cx(0,2)
test_qc.cx(1,2)
test_qc.ccx(0,1,3)

# Finally, we will measure the bottom two qubits to extract the output
test_qc.measure(2,0)
test_qc.measure(3,1)
test_qc.draw()

┌───┐                     
q_0: ┤ X ├──■─────────■────────
     ├───┤  │         │        
q_1: ┤ X ├──┼────■────■────────
     └───┘┌─┴─┐┌─┴─┐  │  ┌─┐   
q_2: ─────┤ X ├┤ X ├──┼──┤M├───
          └───┘└───┘┌─┴─┐└╥┘┌─┐
q_3: ───────────────┤ X ├─╫─┤M├
                    └───┘ ║ └╥┘
c: 2/═════════════════════╩══╩═
                          0  1

In [ ]:
job = sim.run(test_qc)  # run the experiment
result = job.result()   # get the results
result.get_counts()     # interpret the results as a “counts” dictionary

{'10': 1024}